In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

In [4]:
# Helper function to load the raw inertial signals
def load_ucihar(data_dir='har_dataset', subset='train'): 
    """
    Loads the UCI HAR data from the Inertial Signals folder.
    Returns:
        X: numpy array of shape (num_samples, seq_len, num_signals)
        y: numpy array of labels (0-indexed)
    """

    # The nine signal types available in the dataset
    signal_types = [
        "body_acc_x", "body_acc_y", "body_acc_z",
        "body_gyro_x", "body_gyro_y", "body_gyro_z",
        "total_acc_x", "total_acc_y", "total_acc_z"
    ]
    signals = []

    # Each signal file is located in {data_dir}/{subset}/Inertial Signals/ 
    for signal in signal_types:
        filename = os.path.join(data_dir, subset, "Inertial Signals", f"{signal}_{ subset}.txt")
        # Each file has shape (num_samples, 128)
        data = np.loadtxt(filename)
        # Add a new axis so that we can later stack to shape (num_samples, 128, num_signals)
        signals.append(data[..., np.newaxis])
        
    # Stack along the last dimension to form (num_samples, 128, 9)
    X = np.concatenate(signals, axis=2)
    # Load labels from y_{subset}.txt; labels in the dataset are 1-indexed, so subtract 1.
    y_path = os.path.join(data_dir, subset, f"y_{subset}.txt") 
    y = np.loadtxt(y_path).astype(int) - 1

    return X, y

In [5]:
# Define a PyTorch Dataset for UCI HAR
class UCIHARDataset(Dataset):
    def __init__(self, data_dir='har_dataset', subset='train'):
        self.X, self.y = load_ucihar(data_dir, subset) 
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        sample = torch.tensor(self.X[idx], dtype=torch.float32) 
        label = torch.tensor(self.y[idx], dtype=torch.long) 
        return sample, label

In [6]:
SEED = 42
random.seed(SEED) 
np.random.seed(SEED) 
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [7]:
# define model
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)

        logit = self.fc(out[:, -1, :])
        prob = nn.functional.softmax(logit, dim=1) 
        return prob, logit

In [8]:
num_epochs, batch_size, lr = 10, 16, 0.001 
# create train, test dataset 
train_dataset, test_dataset = UCIHARDataset(), UCIHARDataset(data_dir='har_dataset', subset='test')

# create train, test loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:

# create model, loss criterion, optimizer
def train(model, train_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    num_epochs = 10

    # start training
    for epoch in range(num_epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            activity_prob, activity_logit = model(inputs)
            loss = criterion(activity_logit, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}')

# evaluate on test data
def eval(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():  
        for inputs, labels in test_loader:
            activity_prob, activity_logit = model(inputs)
            _, predicted = torch.max(activity_logit, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    test_acc = 100 * correct / total
    print(f'Test Accuracy: {test_acc:.2f}%\n')


## RNN

In [10]:
model = RNNClassifier(9, 16, 1, 6)
train(model, train_loader)
eval(model, test_loader)

Epoch 1/10, Loss: 1.3305
Epoch 2/10, Loss: 1.0995
Epoch 3/10, Loss: 1.2792
Epoch 4/10, Loss: 1.1664
Epoch 5/10, Loss: 0.9955
Epoch 6/10, Loss: 0.8674
Epoch 7/10, Loss: 0.8602
Epoch 8/10, Loss: 0.6662
Epoch 9/10, Loss: 0.7107
Epoch 10/10, Loss: 0.3737
Test Accuracy: 66.20%



In [11]:
model_configs = (
    [64, 1],
    [16, 2],
    [16, 3],
    [16, 4]
) # [feature dims, RNN layer config per model]

for model_config in model_configs:
    feat_dims = model_config[0]
    rnn_layers = model_config[1]
    print(f'------ Model config ---------\nFeature Dims: {feat_dims}\nRNN Layers: {rnn_layers}\n')
    model = RNNClassifier(9, feat_dims, rnn_layers, 6)
    train(model, train_loader)
    eval(model, test_loader)

------ Model config ---------
Feature Dims: 64
RNN Layers: 1

Epoch 1/10, Loss: 1.5719
Epoch 2/10, Loss: 1.0291
Epoch 3/10, Loss: 1.2365
Epoch 4/10, Loss: 0.7015
Epoch 5/10, Loss: 1.1045
Epoch 6/10, Loss: 0.7875
Epoch 7/10, Loss: 0.9233
Epoch 8/10, Loss: 0.4915
Epoch 9/10, Loss: 1.0271
Epoch 10/10, Loss: 0.8041
Test Accuracy: 58.40%

------ Model config ---------
Feature Dims: 16
RNN Layers: 2

Epoch 1/10, Loss: 0.8847
Epoch 2/10, Loss: 1.1040
Epoch 3/10, Loss: 0.7165
Epoch 4/10, Loss: 0.7701
Epoch 5/10, Loss: 0.6416
Epoch 6/10, Loss: 0.4307
Epoch 7/10, Loss: 0.4618
Epoch 8/10, Loss: 0.8093
Epoch 9/10, Loss: 0.3892
Epoch 10/10, Loss: 0.3397
Test Accuracy: 70.21%

------ Model config ---------
Feature Dims: 16
RNN Layers: 3

Epoch 1/10, Loss: 0.8714
Epoch 2/10, Loss: 0.8941
Epoch 3/10, Loss: 0.4600
Epoch 4/10, Loss: 0.6518
Epoch 5/10, Loss: 0.8868
Epoch 6/10, Loss: 1.4315
Epoch 7/10, Loss: 1.3051
Epoch 8/10, Loss: 0.3785
Epoch 9/10, Loss: 0.6869
Epoch 10/10, Loss: 0.8038
Test Accuracy: 

## LSTM

In [12]:
# define model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))

        logit = self.fc(out[:, -1, :])
        prob = nn.functional.softmax(logit, dim=1) 
        return prob, logit

In [13]:
lstm_model = LSTMClassifier(9, 16, 1, 6)
train(lstm_model, train_loader)
eval(lstm_model, test_loader)

Epoch 1/10, Loss: 0.8308
Epoch 2/10, Loss: 0.7573
Epoch 3/10, Loss: 0.9769
Epoch 4/10, Loss: 0.3603
Epoch 5/10, Loss: 1.1496
Epoch 6/10, Loss: 1.1740
Epoch 7/10, Loss: 0.8666
Epoch 8/10, Loss: 0.3781
Epoch 9/10, Loss: 0.2382
Epoch 10/10, Loss: 1.1934
Test Accuracy: 65.22%

